<a href="https://colab.research.google.com/github/diegotluz/face_biometria/blob/main/face_recognation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nova seção

In [4]:
#@title Instalação de dependências
!pip install mediapipe scikit-learn

In [6]:
#@title Importações necessárias
from IPython.display import display, Javascript
import cv2
import mediapipe as mp
import numpy as np
import pickle
import time
import os
import base64
from google.colab.output import eval_js
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline



In [7]:
#@title Configurações globais
KNOWN_FACES_PATH = "./known_faces"
DATABASE_PATH = "face_database.pkl"
FPS = 30
SIMILARITY_THRESHOLD = 0.75
MIN_BLINKS = 3

In [15]:
#@title Classe Principal do Sistema Biométrico
class FaceBiometricSystem:
    def __init__(self):
        self.LANDMARK_INDICES = [33, 133, 362, 263, 61, 291, 4, 168, 197, 195]
        self.EYE_INDICES = {
            'left': [362, 385, 387, 263, 373, 380],
            'right': [33, 160, 158, 133, 153, 144]
        }

        self.face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )

        self.database = self._initialize_database()
        self._validate_database()
        self.scaler = StandardScaler()
        self.clf = self._train_model()
        self.blink_counter = 0
        self.last_blink_time = time.time()

    #@title Métodos de Gerenciamento de Banco de Dados
    def _initialize_database(self):
        if os.path.exists(DATABASE_PATH):
            return self._load_database()
        return self._create_database()

    def _load_database(self):
        try:
            with open(DATABASE_PATH, 'rb') as f:
                return pickle.load(f)
        except:
            return self._create_database()

    def _save_database(self, database):
        with open(DATABASE_PATH, 'wb') as f:
            pickle.dump(database, f)


    def _create_database(self):
        database = {}
        for person in os.listdir(KNOWN_FACES_PATH):
            person_dir = os.path.join(KNOWN_FACES_PATH, person)
            if os.path.isdir(person_dir):
                embeddings = []
                for img_file in os.listdir(person_dir):
                    img_path = os.path.join(person_dir, img_file)
                    img = cv2.imread(img_path)
                    if img is not None:
                        emb = self._extract_embedding(img)
                        if emb is not None:
                            embeddings.append(emb)
                if embeddings:
                    database[person] = {
                        'embeddings': embeddings,
                        'avg_embedding': np.mean(embeddings, axis=0),
                        'samples': len(embeddings)
                    }
        self._save_database(database)
        return database

    #@title Validação de Dados
    def _validate_database(self):
        if len(self.database) < 2:
            raise ValueError("Cadastre pelo menos 2 pessoas no diretório 'known_faces'")

        for person, data in self.database.items():
            if len(data['embeddings']) < 3:
                raise ValueError(f"{person} precisa de pelo menos 3 fotos para treinamento")

    #@title Processamento Facial
    def _extract_embedding(self, image):
        image = cv2.resize(image, (320, 240))
        results = self.face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks:
            return self._process_landmarks(results.multi_face_landmarks[0].landmark)
        return None

    def _process_landmarks(self, landmarks):
        return np.array([(landmarks[i].x, landmarks[i].y) for i in self.LANDMARK_INDICES]).flatten()

    #@title Treinamento do Modelo
    def _train_model(self):
        X = []
        y = []
        persons = list(self.database.keys())

        # Geração de pares positivos
        for person in persons:
            embeddings = self.database[person]['embeddings']
            for i in range(len(embeddings)):
                for j in range(i+1, len(embeddings)):
                    X.append(self._hybrid_metric(embeddings[i], embeddings[j]))
                    y.append(1)

        # Geração de pares negativos
        for i in range(len(persons)):
            for j in range(i+1, len(persons)):
                emb1 = self.database[persons[i]]['embeddings']
                emb2 = self.database[persons[j]]['embeddings']
                for e1 in emb1[:3]:
                    for e2 in emb2[:3]:
                        X.append(self._hybrid_metric(e1, e2))
                        y.append(0)

        X = np.array(X).reshape(-1, 1)
        self.scaler.fit(X)
        X_scaled = self.scaler.transform(X)

        return GradientBoostingClassifier(
            n_estimators=100,
            max_depth=3,
            subsample=0.8,
            random_state=42
        ).fit(X_scaled, y)

    #@title Autenticação e Verificação
    def authenticate(self, frame):
        results = self.face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            return "Desconhecido", 0.0, False

        landmarks = results.multi_face_landmarks[0].landmark
        emb = self._process_landmarks(landmarks)

        best_match = "Desconhecido"
        best_score = 0.0

        for person, data in self.database.items():
            for db_emb in data['embeddings']:
                features = self._hybrid_metric(emb, db_emb)
                features = np.array([features]).reshape(-1, 1)
                features = self.scaler.transform(features)
                score = self.clf.predict_proba(features)[0][1]
                if score > best_score and score > SIMILARITY_THRESHOLD:
                    best_match = person
                    best_score = score

        liveness = self._detect_liveness(landmarks)
        return best_match, best_score, liveness

    #@title Métodos Auxiliares
    def _hybrid_metric(self, emb1, emb2):
        return np.linalg.norm(emb1 - emb2)

    def _eye_aspect_ratio(self, landmarks, eye_type):
        points = [(landmarks[i].x, landmarks[i].y) for i in self.EYE_INDICES[eye_type]]
        vertical = np.linalg.norm(np.subtract(points[1], points[5])) + np.linalg.norm(np.subtract(points[2], points[4]))
        horizontal = np.linalg.norm(np.subtract(points[0], points[3]))
        return vertical / (2.0 * horizontal + 1e-6)

    def _detect_liveness(self, landmarks):
        left_ear = self._eye_aspect_ratio(landmarks, 'left')
        right_ear = self._eye_aspect_ratio(landmarks, 'right')
        ear = (left_ear + right_ear) / 2.0

        current_time = time.time()
        if ear < 0.21:
            if (current_time - self.last_blink_time) > 0.3:
                self.blink_counter += 1
                self.last_blink_time = current_time
        else:
            if (current_time - self.last_blink_time) > 2.0:
                self.blink_counter = 0  # Reset counter if no blinks for 2 seconds

        return self.blink_counter >= MIN_BLINKS

    #@title Interface de Vídeo
    def _video_stream(self):
        js = Javascript('''
            var video;
            var div = null;
            var stream;
            var captureCanvas;
            var imgElement;
            var labelElement;

            var pendingResolve = null;
            var shutdown = false;

            function removeDom() {
                if(stream) stream.getVideoTracks()[0].stop();
                if(video) video.remove();
                if(div) div.remove();
                video = null;
                div = null;
                stream = null;
                imgElement = null;
                captureCanvas = null;
                labelElement = null;
            }

            function onAnimationFrame() {
                if (!shutdown) {
                    window.requestAnimationFrame(onAnimationFrame);
                }
                if (pendingResolve) {
                    captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                    var result = shutdown ? "" : captureCanvas.toDataURL('image/jpeg', 0.8);
                    pendingResolve(result);
                    pendingResolve = null;
                }
            }

            async function createDom() {
                if (div !== null) return stream;

                div = document.createElement('div');
                div.style.border = '2px solid black';
                div.style.padding = '3px';
                div.style.width = '100%';
                div.style.maxWidth = '600px';
                document.body.appendChild(div);

                const modelOut = document.createElement('div');
                modelOut.innerHTML = "<span>Status:</span>";
                labelElement = document.createElement('span');
                labelElement.innerText = 'Inicializando...';
                labelElement.style.fontWeight = 'bold';
                modelOut.appendChild(labelElement);
                div.appendChild(modelOut);

                video = document.createElement('video');
                video.style.display = 'block';
                video.width = div.clientWidth - 6;
                video.setAttribute('playsinline', '');
                video.onclick = () => shutdown = true;
                stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "user"}});
                div.appendChild(video);

                imgElement = document.createElement('img');
                imgElement.style.position = 'absolute';
                imgElement.style.zIndex = 1;
                imgElement.onclick = () => shutdown = true;
                div.appendChild(imgElement);

                const instruction = document.createElement('div');
                instruction.innerHTML = '<span style="color: red; font-weight: bold;">Clique na tela ou pressione Q para sair</span>';
                div.appendChild(instruction);
                instruction.onclick = () => shutdown = true;

                video.srcObject = stream;
                await video.play();

                captureCanvas = document.createElement('canvas');
                captureCanvas.width = 640;
                captureCanvas.height = 480;
                window.requestAnimationFrame(onAnimationFrame);

                return stream;
            }

            async function stream_frame(label, imgData) {
                if (shutdown) {
                    removeDom();
                    return '';
                }

                await createDom();
                labelElement.innerHTML = label;

                if (imgData) {
                    var videoRect = video.getClientRects()[0];
                    imgElement.style.top = videoRect.top + "px";
                    imgElement.style.left = videoRect.left + "px";
                    imgElement.style.width = videoRect.width + "px";
                    imgElement.style.height = videoRect.height + "px";
                    imgElement.src = imgData;
                }

                var result = await new Promise(resolve => pendingResolve = resolve);
                return {'img': result};
            }
        ''')
        display(js)

    #@title Execução Principal
    def run(self):
        self._video_stream()

        while True:
            try:
                frame_data = eval_js('stream_frame("", "")')
                frame = cv2.imdecode(np.frombuffer(
                    base64.b64decode(frame_data['img'].split(',')[1]),
                    dtype=np.uint8), 1)

                # Processar frame e desenhar retângulo
                results = self.face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                if results.multi_face_landmarks:
                    landmarks = results.multi_face_landmarks[0].landmark
                    x_coords = [landmarks[i].x for i in self.LANDMARK_INDICES]
                    y_coords = [landmarks[i].y for i in self.LANDMARK_INDICES]

                    x_min, x_max = int(min(x_coords)*frame.shape[1]), int(max(x_coords)*frame.shape[1])
                    y_min, y_max = int(min(y_coords)*frame.shape[0]), int(max(y_coords)*frame.shape[0])

                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                    # Converter frame para exibição
                    _, img_encoded = cv2.imencode('.jpg', frame)
                    img_base64 = base64.b64encode(img_encoded).decode('utf-8')

                    name, score, liveness = self.authenticate(frame)
                    status = f"Usuário: {name} | Confiança: {score:.2f} | Blinks: {self.blink_counter}"
                    eval_js(f'stream_frame("{status}", "data:image/jpeg;base64,{img_base64}")')

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    eval_js('shutdown = true')
                    break

            except Exception as e:
                print(f"Erro: {str(e)}")
                break

In [ ]:
#@title Inicialização do Sistema
if __name__ == "__main__":
    if os.path.exists(DATABASE_PATH):
        os.remove(DATABASE_PATH)
    system = FaceBiometricSystem()
    system.run()